# Word2vec preprocessing
Preprocessing is not the most interesting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/textdata). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parced from the internet)
1. tokenization
1. building the vocabulary and choosing its size
1. assigning each token a number (numericalization)
1. data structuring and batching

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpfull to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `(batch_size, 2*window_size)`, `(batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(batch_size, window_size, ...), SkipGram(num_skips, skip_window). You should implement only one batcher in this task, it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in separate file. It will be reused for the next task. Result of your work should represent that your batch have proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
bag_window = 2

batch = [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

labels = ['against', 'early', 'working', 'class']
```

If you struggle with somethng, ask your neighbour. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - just do it! Good luck!

# Download words

In [0]:
import collections
import math
import os
import random
import zipfile
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim 
import torch.nn.functional
from torch.optim.lr_scheduler import StepLR
import time

import numpy as np
from six.moves import urllib
from six.moves import xrange
import tensorflow as tf

In [0]:
def maybe_download(filename,url, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('ОК', filename)
    return filename
url = 'http://mattmahoney.net/dc/'
filename = maybe_download('text8.zip',url, 31344016)

ОК text8.zip


In [0]:
 # Read the data into a list of strings.
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
words = read_data(filename) # набор всех слов
print('Data size = ', len(words))

Data size =  17005207


# Building dictionary + batching

In [0]:
UNK_TOKEN = '<TOKEN>'# для редких слов
frequency = collections.Counter(words)
freq = frequency.most_common(11000)# словарь частых слов
low_bound = freq[-1][1]
vocab = [x[0] for x in freq]
vocab.append(UNK_TOKEN)
data = []
for i, word in enumerate(words):
  if frequency[word] > low_bound:
    data.append(word)
  else:
    data.append(UNK_TOKEN)

In [0]:
word_index = {w: idx for (idx, w) in enumerate(vocab)}
index_word = {idx: w for (idx, w) in enumerate(vocab)}

In [0]:
data_idx = 0
class Batcher:
  def __init__(self, batch_size, window_size, data, data_idx=0):
    self.batch_size = batch_size
    self.window_size = window_size
    self.data = data

  def __next__(self, data_idx_=None):
    window_size = self.window_size
    batch_size = self.batch_size
    data = self.data
    global data_idx
    span = 2 * window_size + 1
    batch = []
    labels = [] 
  
    context = collections.deque(maxlen= span)
    for _ in range(2 * window_size + 1):
      context.append(word2idx[data[data_idx]])
      data_idx = (data_idx + 1) % len(data)
  
    for i in range(batch_size):
      batch.append([context[j] for j in list(range(window_size))+ list(range(window_size + 1, span))])
      labels.append(context[window_size])
      context.append(word2idx[data[data_idx]])
      data_idx = (data_idx + 1) % len(data)
    return batch, labels
  def __iter__(self):
    return self

In [0]:
def one_hot(batch, voc_size):
  batch_onehot = torch.zeros(len(batch), voc_size)
  for i, x in enumerate(batch):
    batch_onehot[i, x] = 1
  return batch_onehot